In [23]:
import chardet

chardet.detect(open("../data/train_val_data.csv",'rb').read(400000))

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [15]:
df = pd.read_csv("../data/train_val_data.csv")
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,61900,54,2,2,4,12,2,2,2.000000,0.000000,...,24,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,24800,49,2,0,4,0,2,2,2.000000,0.000000,...,24,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,443,5885346,9,6,710,3826,517,0,78.888889,172.656773,...,32,292949.0,0.0,292949,292949,5592381.0,0.0,5592381,5592381,BENIGN
3,49738,57,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,80,98306933,6,7,363,11595,345,0,60.500000,139.406958,...,20,14018.0,0.0,14018,14018,98300000.0,0.0,98300000,98300000,DoS Hulk


In [16]:
df.shape

(79998, 79)

In [17]:
# drop duplicates
df = df.drop_duplicates()
df = df.dropna()

df.shape

(77340, 79)

In [20]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()

df.shape

(77305, 79)

In [24]:
# columns to keep (featiure engineering)

# List of features to keep
keep_features = [
    'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
    'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std',
    'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
    'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
    'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
    'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
    'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count',
    'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags',
    'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'Down/Up Ratio', 'Average Packet Size',
    'Label'  # Keep the target
]


# Keep only the selected columns
df = df[keep_features]

df.shape


(77305, 45)

In [25]:
# combine rare attacks

# Define rare attacks to combine
rare_attacks = ['Bot', 'Web Attack � Brute Force', 'Web Attack � XSS']

# Create a new column or overwrite Label
df['Label'] = df['Label'].apply(lambda x: 'Other_Attack' if x in rare_attacks else x)

# Check the new class distribution
print(df['Label'].value_counts())

BENIGN              63101
DoS Hulk             5162
PortScan             4373
DDoS                 3618
DoS GoldenEye         291
FTP-Patator           189
DoS slowloris         162
DoS Slowhttptest      155
SSH-Patator           138
Other_Attack          116
Name: Label, dtype: int64


In [26]:
df.shape

(77305, 45)

In [30]:
# is there a reasonabke order??? maybe


# Create encoder
le = LabelEncoder()

# Fit and transform the Label column
df['Label'] = le.fit_transform(df['Label'])

# Optional: see the mapping of classes
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label mapping:", label_mapping)

# Check the new class distribution
print(df['Label'].value_counts())

Label mapping: {'BENIGN': 0, 'DDoS': 1, 'DoS GoldenEye': 2, 'DoS Hulk': 3, 'DoS Slowhttptest': 4, 'DoS slowloris': 5, 'FTP-Patator': 6, 'Other_Attack': 7, 'PortScan': 8, 'SSH-Patator': 9}
0    63101
3     5162
8     4373
1     3618
2      291
6      189
5      162
4      155
9      138
7      116
Name: Label, dtype: int64


In [32]:
# separe features and labels
# X = features, y = target
X = df.drop('Label', axis=1)
y = df['Label']

In [37]:
# train/vasl split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,      # preserves class distribution
    random_state=42
)

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)

Train shape: (61844, 44)
Val shape: (15461, 44)


In [45]:
print(y_train.value_counts())

0    50481
3     4130
8     3498
1     2894
2      233
6      151
5      130
4      124
9      110
7       93
Name: Label, dtype: int64


In [46]:
# smote in here!!!!!!!!
smote = SMOTE(sampling_strategy=0.3, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [47]:
print(y_train.value_counts())

0    50481
3    50481
1    50481
8    50481
4    50481
5    50481
2    50481
6    50481
9    50481
7    50481
Name: Label, dtype: int64


In [38]:
# for those moldes that needs it: scaling

# Suppose X_train, X_val, X_test are your features
scaler = StandardScaler()

# 1️⃣ Fit only on training data
scaler.fit(X_train)

# 2️⃣ Transform train, validation, and test
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)


In [40]:
# torch datasets

import torch
from torch.utils.data import TensorDataset, DataLoader

# -----------------------------
# 1️⃣ Convert data to PyTorch tensors
# -----------------------------
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_val_tensor = torch.tensor(X_val_scaled.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [48]:
## Putting all together

# combine functions for preprocessing

def preprocessing(path, features, do_smote=True, sampling_strategy=0.3, batch_size=64):
    # load data
    df = pd.read_csv(path)
    # drop duplicates
    df = df.drop_duplicates()
    df = df.dropna()    
    # handle inf
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.dropna()
    # features to keep
    df = df[keep_features]
    # combine rare attacks
    rare_attacks = ['Bot', 'Web Attack � Brute Force', 'Web Attack � XSS']
    df['Label'] = df['Label'].apply(lambda x: 'Other_Attack' if x in rare_attacks else x)
    # label encoding
    le = LabelEncoder()
    df['Label'] = le.fit_transform(df['Label'])
    # separe features and labels
    X = df.drop('Label', axis=1)
    y = df['Label']
    # train/val split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,      # preserves class distribution
        random_state=42
    )
    # smote
    if do_smote:
        smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
        X_train, y_train = smote.fit_resample(X_train, y_train)
    # scale (only for nnets)
    if do_scale:
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
        X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)

    # create dataloaders
    X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    
    X_val_tensor = torch.tensor(X_val_scaled.values, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader
    
    

        
            
    


# smote

✅ Recommended order for your pipeline

Drop nulls / clean data
→ SMOTE can’t handle missing values, so clean first.

Combine rare classes / categories
→ Do this before encoding, so you don’t generate rare dummy variables.

Feature selection
→ Optional before resampling, but ensure you keep the features SMOTE will use.

Label encoding / one-hot encoding
→ Yes, do this before SMOTE.
SMOTE works on numeric data, not strings or categories.

Separate features and labels (X, y)
→ Required for SMOTE.

Train/validation split
⚠️ Important:
Apply SMOTE only on the training set, never before splitting.
Otherwise, synthetic samples will leak into validation data and inflate metrics.

In [41]:
"""
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


# scaling (if needed)
scaler.fit(X_train_res)
X_train_scaled = scaler.transform(X_train_res)
X_val_scaled = scaler.transform(X_val)
"""

'\nfrom sklearn.model_selection import train_test_split\nfrom imblearn.over_sampling import SMOTE\n\nX_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)\n\nsmote = SMOTE(random_state=42)\nX_train_res, y_train_res = smote.fit_resample(X_train, y_train)\n\n\n# scaling (if needed)\nscaler.fit(X_train_res)\nX_train_scaled = scaler.transform(X_train_res)\nX_val_scaled = scaler.transform(X_val)\n'

| Step               | Apply SMOTE before/after?     |
| ------------------ | ----------------------------- |
| Drop nulls         | Before                        |
| Combine rare cases | Before                        |
| Encode categorical | Before                        |
| Split train/val    | Before                        |
| SMOTE              | ✅ After split (only on train) |
| Scaling            | After SMOTE                   |



de (clean → encode → split → SMOTE → scale → train